In [1]:
import numpy as np
import pandas as pd
import pyspark as ps

In [4]:
schema = pd.read_table('twitter-swisscom/schema.txt', header=None, delim_whitespace=True, index_col=0, names=['name', 'type', 'specification', '??', 'format'] )

In [5]:
schema

,name,type,specification,??,format
1,id,bigint(20),UNSIGNED,No,None
2,userId,bigint(20),UNSIGNED,No,None
3,createdAt,timestamp,No,0000-00-00,00:00:00
4,text,text,utf8_unicode_ci,No,None
5,longitude,float,Yes,NaN,NaN
6,latitude,float,Yes,NaN,NaN
7,placeId,varchar(25),utf8_general_ci,Yes,NaN
8,inReplyTo,bigint(20),UNSIGNED,Yes,NaN
9,source,int(10),UNSIGNED,No,None
10,truncated,bit(1),No,None,NaN


In [7]:
long_data = pd.read_table('twitter-swisscom/sample.tsv', 
              sep='\t', engine='c', encoding='utf-8', nrows=9000, 
              header=None, names=schema.name, na_values='\\N')

len(long_data['longitude'].dropna())

1322

In [15]:
pd.read_table('twitter-swisscom/twex.tsv', 
              sep='\t', engine='c', encoding='utf-8', 
              header=None, names=schema.name, na_values='\\N', nrows=9)

,id,userId,createdAt,text,longitude,latitude,placeId,inReplyTo,source,truncated,placeLatitude,placeLongitude,sourceName,sourceUrl,userName,screenName,followersCount,friendsCount,statusesCount,userLocation
0,9514097914,17341045,2010-02-23 05:55:51,Guuuuten Morgen! :-),7.43926,46.9489,NaN,NaN,197,NaN,NaN,NaN,TwitBird,http://www.nibirutech.com,Tilman Jentzsch,blickwechsel,586,508,9016,"Bern, Switzerland"
1,9514846412,7198282,2010-02-23 06:22:40,Still the best coffee in town — at La Stanza h...,8.53781,47.3678,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Nico Luchsinger,halbluchs,1820,703,4687,"Zurich, Switzerland"
2,9516574359,14657884,2010-02-23 07:34:25,It has been a week or so.. and today I just co...,6.13396,46.1951,NaN,NaN,3,NaN,NaN,NaN,foursquare,http://foursquare.com,Javier Belmonte,vichango,167,277,2885,"Geneva, Switzerland"
3,9516952605,14703863,2010-02-23 07:51:47,Getting ready.. http://twitpic.com/14v8gz,8.81749,47.2288,NaN,NaN,62,NaN,NaN,NaN,Twittelator,http://stone.com/Twittelator,Urs,ugro,75,161,1390,"Zürich, Switzerland"
4,9517198943,14393717,2010-02-23 08:02:57,Un peu de réconfort liquide en take away après...,6.63254,46.5199,NaN,NaN,3,NaN,NaN,NaN,foursquare,http://foursquare.com,Romain P.,PIMboula,135,109,2381,"Lausanne, Suisse"
5,9517916537,13535402,2010-02-23 08:35:39,I'm at Online PC Magazin in Adliswil http://go...,8.53010,47.3152,NaN,NaN,550,NaN,NaN,NaN,Gowalla,http://gowalla.com/,Patrick Hediger,hediger,1511,682,12157,"Zurich, Switzerland"
6,9518014755,14393717,2010-02-23 08:40:13,Au charbon! (@ BCV St-François) http://4sq.com...,6.63428,46.5191,NaN,NaN,3,NaN,NaN,NaN,foursquare,http://foursquare.com,Romain P.,PIMboula,135,109,2381,"Lausanne, Suisse"
7,9519149278,14260616,2010-02-23 09:32:09,@eyeem When and how can we send photos ? One p...,8.29953,47.4829,NaN,9.518987e+09,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Roman Keller,RomanKeller,720,821,7337,Switzerland
8,9519737890,14657884,2010-02-23 09:59:41,"Magic spells run off after midnight, I guess s...",6.13870,46.1750,NaN,NaN,1,NaN,NaN,NaN,Twitter for iPhone,http://twitter.com/#!/download/iphone,Javier Belmonte,vichango,167,277,2885,"Geneva, Switzerland"


In [7]:
#spark.read.csv(
#    "twitter-swisscom/sample.tsv", header=False, mode="DROPMALFORMED", schema=schema)

In [1]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
sc = SparkContext('local', 'pyspark')
sqlContext = SQLContext(sc)
df = sqlContext.read.format('com.databricks.spark.csv').options(header='false').load('twitter-swisscom/sample.tsv')
df.show()

+--------------------+
|                 _c0|
+--------------------+
|77652298383795404...|
|77652300063620301...|
|77652304520069120...|
|77652305840429056...|
|77652305850492518...|
|77652307102501273...|
|77652309276821913...|
|77652310500717772...|
|77652312672947404...|
|77652313440924467...|
|77652315035189248...|
|77652315612322611...|
|77652316603432550...|
|77652316736389939...|
|77652318896037068...|
|77652320516298342...|
|77652324701800857...|
|77652325676129075...|
|77652326936107417...|
|77652327111839744...|
+--------------------+
only showing top 20 rows



Question:
 - how do we deal with this in Spark uniquely.
 - we will have to prefilter, we can't load everything in frame anyway
 - so best do it in spark, but how? what code will enable us to do this, eg how do we deal with this in spark...as we have it distributed & need to read the tsv file...how will this work.
 - there should be a way to use the schema file to help..but how